In [1]:
import sys

sys.path.append("../")

import sqlite3
import pandas as pd
from functions.env import DB_SCIENCE_PATH_NEW

data_path = "/Users/charlesdedampierre/Desktop/cultura_project/cultura.db"

conn = sqlite3.connect(data_path)
new_conn = sqlite3.connect(DB_SCIENCE_PATH_NEW)


In [19]:
df_ind = pd.read_sql("SELECT * FROM individual_occupations", conn)
df_ind[df_ind['individual_wikidata_id']=='Q913']

#df_ind[df_ind['occupations_name']=='philosopher']

,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category
29689,Q913,Socrates,Q4964182,philosopher,writer


In [37]:
df_ind[df_ind['occupations_name']=='theologian']

,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category
9,Q2822993,Achard of Saint-Victor,Q1234713,theologian,writer
23,Q1349852,Jakob Wilhelm Feuerlein,Q1234713,theologian,writer
36,Q12319939,Johan Ludvig Brabrand,Q1234713,theologian,writer
86,Q95291648,Karl Stern,Q1234713,theologian,writer
88,Q38541085,Bésian Arroy,Q1234713,theologian,writer
...,...,...,...,...,...
319432,Q3939246,Roberto de' Bardi,Q1234713,theologian,writer
319443,Q434550,Jerome of Prague,Q1234713,theologian,writer
319447,Q733598,Moses ben Jacob Cordovero,Q1234713,theologian,writer
319484,Q2848882,Andrea da Segni,Q1234713,theologian,writer


In [40]:
test = df_ind.groupby('individual_wikidata_id')['occupations_name'].apply(lambda x : ' | '.join(x))
test = test.reset_index()
test[test['occupations_name']=='philosopher']

,individual_wikidata_id,occupations_name
105,Q1001422,philosopher
497,Q100331569,philosopher
521,Q100362120,philosopher
564,Q1003828,philosopher
718,Q1006982,philosopher
...,...,...
219867,Q99355945,philosopher
219988,Q99493558,philosopher
220138,Q99585050,philosopher
220310,Q99794,philosopher


In [29]:

test = test[test['occupations_name'].str.contains('philosopher')]
test['occupations_name'] = test['occupations_name'].apply(lambda x : x.split(' | '))
test = test.explode('occupations_name')

,individual_wikidata_id,occupations_name
105,Q1001422,philosopher
172,Q100210,anthropologist
172,Q100210,philosopher
250,Q100288,pedagogue
250,Q100288,philosopher
...,...,...
219905,Q994181,historian
219988,Q99493558,philosopher
220138,Q99585050,philosopher
220310,Q99794,philosopher


In [32]:
test.occupations_name.value_counts().head(20)

philosopher           5937
theologian             840
university teacher     798
poet                   631
historian              440
mathematician          420
translator             294
astronomer             209
physicist              165
linguist               147
economist              129
pedagogue              112
professor              110
literary critic         99
philologist             88
playwright              68
naturalist              66
botanist                59
essayist                57
composer                57
Name: occupations_name, dtype: int64

,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category


In [2]:


df_ind_science = df_ind[df_ind["occupations_category"].str.contains("science")]
list_individuals = list(set(df_ind_science.individual_wikidata_id))

df_ind = df_ind[df_ind['individual_wikidata_id'].isin(list_individuals)]

print(len(set(df_ind.individual_wikidata_id)))

df_ind[df_ind['occupations_name']=='philosopher']


42668


,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category
179,Q675,André-Marie Ampère,Q4964182,philosopher,writer
1674,Q201221,Alexander Ivanovich Herzen,Q4964182,philosopher,writer
1819,Q947485,"Victor de Riqueti, marquis de Mirabeau",Q4964182,philosopher,writer
2162,Q298860,Hippasus,Q4964182,philosopher,writer
2166,Q94840288,David Asher,Q4964182,philosopher,writer
...,...,...,...,...,...
318643,Q166876,Isidore of Seville,Q4964182,philosopher,writer
318691,Q1685927,Jean-Guillaume-César-Alexandre-Hippolyte Colins,Q4964182,philosopher,writer
318853,Q15970301,Julie Favre,Q4964182,philosopher,writer
319225,Q6934061,Mulla Mahmud Jaunpuri,Q4964182,philosopher,writer


In [9]:
df_ind[df_ind['individual_name'].str.contains('Socrates')]

,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category


In [6]:
df_ind[df_ind['individual_wikidata_id']=='Q913']

,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category


In [10]:
df_ind_regions = pd.read_sql("SELECT * FROM individuals_regions", conn)
df_ind_regions = df_ind_regions.drop("individual_name", axis=1)

df_ind_regions = pd.merge(df_ind_regions, df_ind, on = 'individual_wikidata_id')
df_ind_regions = df_ind_regions[
    ["individual_wikidata_id", "individual_name", "region_code", "region_name"]
].drop_duplicates()

In [8]:
print(len(set(df_ind_regions.individual_wikidata_id)))

df_ind_regions.to_sql("individuals_regions", new_conn, if_exists="replace", index=False)

31089


109580

In [9]:
df_ind_birth = pd.read_sql("SELECT * FROM individuals_main_information", conn)
df_ind_birth = df_ind_birth[["individual_wikidata_id", "birthyear"]]
df_ind_birth["productive_year"] = df_ind_birth["birthyear"] + 35

df_ind_final = pd.merge(df_ind, df_ind_birth, on="individual_wikidata_id")

print(len(set(df_ind_final.individual_wikidata_id)))

df_ind_final.to_sql(
    "individuals_occupation_information", new_conn, if_exists="replace", index=False
)


42668


76865